In [5]:
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, LineString, MultiPolygon
from shapely.ops import split
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import copy
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

In [3]:
burned = gpd.read_file("../data/burned/sampleData.csv")
unburned = gpd.read_file("../data/unburned/sampleData.csv")

In [7]:
temp1 = gpd.GeoDataFrame()
temp2 = gpd.GeoDataFrame()
for i in np.unique(burned['FIRE_NAME']):
    temp1 = pd.concat([temp1,burned[burned['FIRE_NAME']==i]])
    temp2 = pd.concat([temp2,unburned[unburned['FIRE_NAME']==i]])

In [8]:
df = pd.concat([temp1,temp2])

In [ ]:
df = pd.concat([temp1,temp2])
classes = {1: "other",       # open water / perennial ice+snow
           2: "developed",
           3: "other",       # barren land
           4: "forest",
           5: "shrub",
           7: "grassland/herbaceous",
           8: "agriculture",
           9: "other"}       # wetland
 
df['landcover'] = df["landcover"].apply(lambda x: classes[int(x)//10])

In [ ]:
df['landcover']

In [9]:
clean_df = df[['burnSeverity','SR_B3','SR_B6','NDVI','elevation','landcover','percent_tree_cover','x_coord','y_coord']]
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
clean_df

<ipython-input-9-4368b2bd5d31>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')


,burnSeverity,SR_B3,SR_B6,NDVI,elevation,landcover,percent_tree_cover,x_coord,y_coord
0,2,9461.0,15058,0.212274,168,52,41,-185709.062502,35992.187398
1,1,9716.0,15731,0.270679,138,71,13,-185825.937502,35843.437398
2,1,9104.0,13635,0.244286,129,43,29,-185825.937502,37681.562398
3,1,8645.0,11935,0.239246,102,43,32,-185857.812502,35630.937398
4,1,9706.0,14894,0.229572,165,71,0,-185868.437502,36310.937398
...,...,...,...,...,...,...,...,...,...
165052,1,8334.0,9964,0.254991,1624,42,68,-61923.125001,230745.624897
165053,1,8884.0,11546,0.307321,1726,42,67,-61923.125001,231199.374897
165054,1,10292.0,9097,0.245669,2078,42,60,-61923.125001,235131.874897
165055,0,29709.0,9832,-0.040084,2296,52,0,-61923.125001,236190.624897


In [10]:
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = clean_df.drop(['burnSeverity'],axis=1)
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
X_scaled

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


array([[ 4.09555663e-01,  9.80646988e-01,  5.04573752e-01, ...,
         1.11117156e-01, -8.72913561e-02, -1.47025787e+00],
       [ 6.27677398e-01,  1.22420023e+00,  1.12733938e+00, ...,
        -8.22921042e-01, -8.86269727e-02, -1.47134951e+00],
       [ 1.04185235e-01,  4.65674827e-01,  8.45917045e-01, ...,
        -2.89184929e-01, -8.86269727e-02, -1.45785993e+00],
       ...,
       [ 1.12037590e+00, -1.17659057e+00,  8.60658554e-01, ...,
         7.44928790e-01,  1.32730160e+00, -8.81657583e-03],
       [ 1.77292768e+01, -9.10600035e-01, -2.18633244e+00, ...,
        -1.25658163e+00,  1.32730160e+00, -1.04664816e-03],
       [-6.94739079e-01, -1.00469193e+00,  1.35087582e+00, ...,
         1.07851386e+00,  1.32588742e+00, -1.04174779e-01]])

In [11]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LogisticRegression()
pipe.fit(x_train, y_train)
print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_[0]
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Logistic Regression: 0.7537278622910366
SR_B3: 3.150200487348103
SR_B6: -3.51634777256248
NDVI: 4.362382716683548
elevation: -0.12828737014299596
landcover: -0.20783207907246032
percent_tree_cover: -2.687288523003459
x_coord: 0.7469564247084589
y_coord: 0.19686400559132


/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['SR_B3', 'NDVI', 'x_coord', 'y_coord']

In [12]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = MLPClassifier(random_state=1, max_iter=300).fit(x_train, y_train)
clf.score(x_test, y_test)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8297296077280729

In [13]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8583036157631021

In [14]:
y_pred = clf.predict(x_test)

In [15]:
confusion_matrix(y_test, y_pred)

array([[  457,   536,     3,     2,     0],
       [   85, 38826,  2155,    62,     0],
       [    7,  3204, 11136,  1077,     1],
       [    0,    82,  1621,  6340,   103],
       [    0,     2,     8,   469,   283]])

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
precision = dict()
recall = dict()
average_precision = dict()
for i in range(5):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], clf_score[:, i])
    average_precision[i] = average_precision_score(y_test[:, i], clf_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(
    y_test.ravel(), clf_score.ravel()
)
average_precision["micro"] = average_precision_score(y_test, clf_score, average="micro")
display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot()
_ = display.ax_.set_title("Micro-averaged over all classes")

In [ ]:
importance = clf.feature_importances_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

In [ ]:
rfc = RandomForestClassifier() 

param_grid = { 
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_train, y_train)
print(CV_rfc.best_params_)

In [ ]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = RandomForestClassifier(max_depth= 10, min_samples_split= 2, n_estimators= 500)
clf.fit(x_train, y_train)
clf.score(x_test, y_test)